In [1]:
!pip install pyomo -q
!pip install mistralai -q
!wget -N -q "https://matematica.unipv.it/gualandi/solvers/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.6 MB/s eta 0:00:00


In [2]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from IPython.display import display, Markdown, Latex
from datetime import datetime

In [3]:
API_KEY = ''

MODEL_ID = 'open-mixtral-8x22b'
MODEL_SEED = 1
MODEL_TEMPERATURE = 0.7

SYSTEM_PROMPT_1 = """Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."""
SYSTEM_PROMPT_2 = """Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."""
PROBLEM_DESCRIPTION = """You are a chemist in charge of a process that requires two liquids to reach the end product. Your goal is to maximize the quality of the product in the end, which is measured by the difference between the Napierian logarithm of the product of the litres used of each component and the cube root of the summation of the first component litres as well as double the second component litres. It is required that the squared difference between the litres employed of each component and the mean litres utilised of both components is below ten. This is necessary for the obtained set to be stable. Furthermore, the mixture tank available to you can only fit 80 l. Please formulate a mathematical optimization model for this problem."""


client = MistralClient(api_key=API_KEY)

print(f'Time of execution: {datetime.now()}')

Time of execution: 2024-06-14 13:51:09.296819


## Step 1 - Generate Mathematical Formulation

In [4]:
messages_1 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION)
]

In [5]:
response_1 = client.chat(
    model=MODEL_ID,
    messages=messages_1,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_1_text = response_1.choices[0].message.content

In [6]:
Markdown(response_1_text)

Parameters:
- Let x1 represent the litres of the first liquid component.
- Let x2 represent the litres of the second liquid component.
- The available tank capacity is denoted as C = 80 litres.

Decision Variables:
- x1 and x2 are the decision variables.

Objective Function:
The quality of the product is denoted as Q, and we want to maximize Q. The formula for Q is given by:

Q = ln(x1 * x2) - (x1 + 2 * x2)^(1/3)

Constraints:
1. The squared difference between the litres used of each component and the mean litres utilized of both components should be less than or equal to 10. This can be represented as:

(x1 - (x1 + x2)/2)^2 ≤ 10
(x2 - (x1 + x2)/2)^2 ≤ 10

2. The total litres used from both components should not exceed the tank capacity:

x1 + x2 ≤ C

Now, the complete optimization model can be formulated as:

Maximize:
Q = ln(x1 * x2) - (x1 + 2 * x2)^(1/3)

Subject to the constraints:
(x1 - (x1 + x2)/2)^2 ≤ 10
(x2 - (x1 + x2)/2)^2 ≤ 10
x1 + x2 ≤ C
x1 ≥ 0, x2 ≥ 0 (non-negativity constraints)

In [7]:
print(response_1_text)

Parameters:
- Let x1 represent the litres of the first liquid component.
- Let x2 represent the litres of the second liquid component.
- The available tank capacity is denoted as C = 80 litres.

Decision Variables:
- x1 and x2 are the decision variables.

Objective Function:
The quality of the product is denoted as Q, and we want to maximize Q. The formula for Q is given by:

Q = ln(x1 * x2) - (x1 + 2 * x2)^(1/3)

Constraints:
1. The squared difference between the litres used of each component and the mean litres utilized of both components should be less than or equal to 10. This can be represented as:

(x1 - (x1 + x2)/2)^2 ≤ 10
(x2 - (x1 + x2)/2)^2 ≤ 10

2. The total litres used from both components should not exceed the tank capacity:

x1 + x2 ≤ C

Now, the complete optimization model can be formulated as:

Maximize:
Q = ln(x1 * x2) - (x1 + 2 * x2)^(1/3)

Subject to the constraints:
(x1 - (x1 + x2)/2)^2 ≤ 10
(x2 - (x1 + x2)/2)^2 ≤ 10
x1 + x2 ≤ C
x1 ≥ 0, x2 ≥ 0 (non-negativity constr

## Step 2 - Generate the Pyomo Code

In [8]:
messages_2 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_2),
    ChatMessage(role="user", content=response_1_text)
]

In [9]:
response_2 = client.chat(
    model=MODEL_ID,
    messages=messages_2,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_2_text = response_2.choices[0].message.content

In [10]:
Markdown(response_2_text)

Here is the Python code using Pyomo to solve the optimization problem:

```python
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Define the parameters
model.C = pyo.Param(initialize=80)  # Tank capacity (litres)

# Define the decision variables
model.x1 = pyo.Var(within=pyo.NonNegativeReals)  # Litres of the first liquid component
model.x2 = pyo.Var(within=pyo.NonNegativeReals)  # Litres of the second liquid component

# Define the objective function
def obj_rule(model):
    return pyo.log(model.x1 * model.x2) - (model.x1 + 2 * model.x2) ** (1/3)

model.Q = pyo.Objective(rule=obj_rule, sense=pyo.maximize)

# Define the constraints
def constraint1_rule(model):
    return (model.x1 - (model.x1 + model.x2) / 2) ** 2 <= 10

model.c1 = pyo.Constraint(rule=constraint1_rule)

def constraint2_rule(model):
    return (model.x2 - (model.x1 + model.x2) / 2) ** 2 <= 10

model.c2 = pyo.Constraint(rule=constraint2_rule)

def constraint3_rule(model):
    return model.x1 + model.x2 <= model.C

model.c3 = pyo.Constraint(rule=constraint3_rule)

# Solve the model
solver = pyo.SolverFactory('ipopt')  # Using IPOPT solver, other solvers can be used as well
solver.solve(model)

# Print the results
print("x1 =", model.x1.value)
print("x2 =", model.x2.value)
print("Q =", model.Q())
```

This code initializes the model, defines the parameters, decision variables, objective function, and constraints. The non-negativity constraints for x1 and x2 are implicitly defined by setting their domains to NonNegativeReals. Then the model is solved using the IPOPT solver, and the results are printed. You can replace the sample data (C = 80) with your own data if you have any.

In [11]:
print(response_2_text)

Here is the Python code using Pyomo to solve the optimization problem:

```python
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Define the parameters
model.C = pyo.Param(initialize=80)  # Tank capacity (litres)

# Define the decision variables
model.x1 = pyo.Var(within=pyo.NonNegativeReals)  # Litres of the first liquid component
model.x2 = pyo.Var(within=pyo.NonNegativeReals)  # Litres of the second liquid component

# Define the objective function
def obj_rule(model):
    return pyo.log(model.x1 * model.x2) - (model.x1 + 2 * model.x2) ** (1/3)

model.Q = pyo.Objective(rule=obj_rule, sense=pyo.maximize)

# Define the constraints
def constraint1_rule(model):
    return (model.x1 - (model.x1 + model.x2) / 2) ** 2 <= 10

model.c1 = pyo.Constraint(rule=constraint1_rule)

def constraint2_rule(model):
    return (model.x2 - (model.x1 + model.x2) / 2) ** 2 <= 10

model.c2 = pyo.Constraint(rule=constraint2_rule)

def constraint3_rule(model):
    return model.x

### Code Executability

In [12]:
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Define the parameters
model.C = pyo.Param(initialize=80)  # Tank capacity (litres)

# Define the decision variables
model.x1 = pyo.Var(within=pyo.NonNegativeReals)  # Litres of the first liquid component
model.x2 = pyo.Var(within=pyo.NonNegativeReals)  # Litres of the second liquid component

# Define the objective function
def obj_rule(model):
    return pyo.log(model.x1 * model.x2) - (model.x1 + 2 * model.x2) ** (1/3)

model.Q = pyo.Objective(rule=obj_rule, sense=pyo.maximize)

# Define the constraints
def constraint1_rule(model):
    return (model.x1 - (model.x1 + model.x2) / 2) ** 2 <= 10

model.c1 = pyo.Constraint(rule=constraint1_rule)

def constraint2_rule(model):
    return (model.x2 - (model.x1 + model.x2) / 2) ** 2 <= 10

model.c2 = pyo.Constraint(rule=constraint2_rule)

def constraint3_rule(model):
    return model.x1 + model.x2 <= model.C

model.c3 = pyo.Constraint(rule=constraint3_rule)

# Solve the model
solver = pyo.SolverFactory('ipopt')  # Using IPOPT solver, other solvers can be used as well
solver.solve(model)

# Print the results
print("x1 =", model.x1.value)
print("x2 =", model.x2.value)
print("Q =", model.Q())

x1 = 43.162277233647174
x2 = 36.8377229631074
Q = 2.4827783318454744


### Solution Correctness